In [53]:
import numpy as np
import pandas as pd
from pandas import DataFrame, Series
from sklearn.metrics.pairwise import cosine_similarity

def get_sim_data(data):
    sim = DataFrame()
    for i in data.columns:
        sim[i] = DataFrame(cosine_similarity([data[guest_id]], [data[i]]))

    return sim.T.sort_values(by=0, ascending=False).head(11).T

def get_rec_data(top10_near):
    mult_rec = DataFrame()
    for i in top10_near.columns:
        if i != guest_id:
            mult_rec[i] = data[i] * (1 + top10_near[i][0])

    return mult_rec.T.sum().sort_values(ascending=False).to_frame('recommend rate').head(10)

guest_id = int(input('insert a user id : '))

data = pd.read_table('data/u.data', sep='\t', usecols=['user id', 'movie id', 'rating'])
data = data.pivot_table(index='movie id', columns='user id', values='rating', fill_value=0)
data = get_rec_data(get_sim_data(data))

item = pd.read_table('data/u.item', sep='|', usecols=['movie id', 'movie title', 'release date'])
item = item.set_index(['movie id'])
pd.concat([data, item], axis=1, join_axes=[data.index])

#data[data[guest_id] > 0][guest_id].to_frame('user id')

insert a user id : 456


,recommend rate,movie title,release date
movie id,,,
56,71.941253,Pulp Fiction (1994),01-Jan-1994
12,69.020342,"Usual Suspects, The (1995)",14-Aug-1995
238,69.018114,Raising Arizona (1987),01-Jan-1987
168,67.554008,Monty Python and the Holy Grail (1974),01-Jan-1974
174,67.553874,Raiders of the Lost Ark (1981),01-Jan-1981
50,66.108102,Star Wars (1977),01-Jan-1977
182,66.080557,GoodFellas (1990),01-Jan-1990
180,66.075831,Apocalypse Now (1979),01-Jan-1979
89,64.655952,Blade Runner (1982),01-Jan-1982
